## Import

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use("Agg")
import seaborn as sns
from astropy.timeseries import LombScargle
from scipy.stats import median_abs_deviation as MAD
from scipy.signal import find_peaks
from tqdm import tqdm

In [36]:
FILENAME = r'data_cleaned/all_mid_time_data_with_OC.csv'
df = pd.read_csv(FILENAME)
df.tail(3)

,Planet,Tmid_(BJD_TDB),Tmid_unc.,source,P_(days),P_unc.,epoch,O-C_(days),O-C_unc._(days)
275545,Kepler-1639b,2.455985e+06,NaN,Holczer+16,9.878925,0.00003,24,0.002083,0.006944
275546,Kepler-1639b,2.456005e+06,NaN,Holczer+16,9.878925,0.00003,26,-0.005556,0.013889
275547,Kepler-1639b,2.456015e+06,NaN,Holczer+16,9.878925,0.00003,27,0.007639,0.013889


## Outlier rejection (5-sigma clipping)

In [37]:
# Remove excessively large TTV error and then 5 sigma clipping
def outlier_reject(df, errthres=3, sigma=5):
    OCerr_mu = df.groupby('Planet')['O-C_unc._(days)'].transform('mean')
    mask_err = df['O-C_unc._(days)'] / OCerr_mu <= errthres
    df_masked = df.loc[mask_err].reset_index(drop=True)
    
    OC_mu = df_masked.groupby('Planet')['O-C_(days)'].transform('mean')
    OC_std = df_masked.groupby('Planet')['O-C_(days)'].transform('std')
    mask_sigma = (df_masked['O-C_(days)'] - OC_mu).abs() <= sigma * OC_std
    df_masked2 = df_masked.loc[mask_sigma].reset_index(drop=True)
    
    return df_masked2
df_clip = outlier_reject(df)

# Choose only remaining rows with at least 5 observations
df_clip = df_clip[df_clip.groupby('Planet')['Planet'].transform('size') >= 5].reset_index(drop=True)

df_clip.tail(3)

,Planet,Tmid_(BJD_TDB),Tmid_unc.,source,P_(days),P_unc.,epoch,O-C_(days),O-C_unc._(days)
272666,Kepler-1639b,2.455985e+06,NaN,Holczer+16,9.878925,0.00003,24,0.002083,0.006944
272667,Kepler-1639b,2.456005e+06,NaN,Holczer+16,9.878925,0.00003,26,-0.005556,0.013889
272668,Kepler-1639b,2.456015e+06,NaN,Holczer+16,9.878925,0.00003,27,0.007639,0.013889


In [38]:
# Import big dataset for later insertion of final parameters
FILENAME = r'data_cleaned/merged_potential_TTV_flagged.csv'
df_big = pd.read_csv(FILENAME)
df_big = df_big[df_big.potential_sinTTV_flag == 1].reset_index(drop=True)
df_big.head(3)

,name_exoplanet.eu,star_name,name_exoclock+holczer,T0_(BJD_TDB),T0_unc.,P_(days),P_unc.,mass,mass_error,radius,...,planet_pos,period_ratio,mass_ratio,potential_sinTTV_flag,TTV_pos,MMR,j,N,Delta,expected_Pttv
0,HAT-P-26 b,HAT-P-26,HAT-P-26b,2.457197e+06,0.00007,4.234501,3.200000e-07,0.058500,0.007170,0.5700,...,1,1.000000,2.844043,1.0,inner,3:2,3.0,1.0,0.038139,57.631633
1,HAT-P-26 d,HAT-P-26,NaN,NaN,NaN,NaN,NaN,0.020569,0.003319,0.1758,...,2,1.557208,1.000000,1.0,outer,3:2,3.0,1.0,0.038139,57.631633
2,HAT-P-27 Ac,HAT-P-27 A,NaN,NaN,NaN,NaN,NaN,0.066608,0.016082,0.3863,...,1,1.000000,1.000000,1.0,inner,5:2,5.0,3.0,0.013699,44.375266


## Periodicity search (GLS)

In [39]:
def lomb_scargle_analysis(df_full, planet_name, expected_Pttv=None):
    # Flags
    flag_outside_search_range = 0
    flag_insufficient_power = 0
    
    # Take only specific planet
    df = df_full[df_full.Planet == planet_name]
    
    # Frequency grid and setup
    Tmid = df['Tmid_(BJD_TDB)']
    Porb = df['P_(days)'].mode().iloc[0]
    OC = df['O-C_(days)']
    OCerr = df['O-C_unc._(days)']

    baseline = Tmid.max() - Tmid.min()
    fmin0, fmax0 = 1 / (10*baseline), 1 / (2*Porb)
    if expected_Pttv == None:
        # For now, return nothing
        # fmin, fmax = fmin0, fmax0
        return
    else:  # If expected Pttv outside search range
        if (expected_Pttv > 1/fmin0) or (expected_Pttv < 1/fmax0):
            flag_outside_search_range = 1
            fmin, fmax = fmin0, fmax0
        else:
            fmin = max(fmin0, 1 / (expected_Pttv * 10**+1.0))
            fmax = min(fmax0, 1 / (expected_Pttv * 10**-1.0))
    
    Nfreq = max(np.int64(10 * baseline * fmax), 5000)  # VanderPlas 2017 for max
    freq = np.linspace(fmin, fmax, Nfreq)

    # Lomb–Scargle (1-harmonic model)
    ls = LombScargle(Tmid, OC, OCerr, nterms=1, normalization="psd")
    power = ls.power(freq, normalization="psd")

    # False Alarm Probability / FAP at 5%, 1%, 0.1%
    fap5 = ls.false_alarm_level(0.05, minimum_frequency=fmin, maximum_frequency=fmax)
    fap1 = ls.false_alarm_level(0.01, minimum_frequency=fmin, maximum_frequency=fmax)
    fap01 = ls.false_alarm_level(0.001, minimum_frequency=fmin, maximum_frequency=fmax)
    fap_threshold = float(np.atleast_1d(fap01)[0])
    
    # Find power peaks with FAP < 0.1%
    peaks_idx, _ = find_peaks(power, height=fap_threshold)
    flag_insufficient_power = 1 if (peaks_idx.size == 0) else 0
    
    if (flag_insufficient_power == 1) or (flag_outside_search_range == 1):
        power_best = power.max()
        idx_best = np.argmax(power)
        f_best = freq[idx_best]

    elif (flag_insufficient_power == 0):
        peak_freqs = freq[peaks_idx]
        peak_periods = 1/peak_freqs
        # Peak at fitted period closest to expected TTV period
        peaks_idx_best = np.argmin(np.abs(peak_periods - expected_Pttv))
        idx_best = peaks_idx[peaks_idx_best]
        f_best = freq[idx_best]
        power_best = power[idx_best]

    power_relative_to_max = power_best / power.max()
    
    # Best frequency determination and their FAP
    model = ls.model(Tmid, f_best)
    fap = ls.false_alarm_probability(power_best, minimum_frequency=fmin, maximum_frequency=fmax)

    # BICs for linear vs sinusoidal determination
    offset = ls.model_parameters(f_best)[0]
    chi2_0 = np.sum(((OC - offset) / OCerr)**2)
    chi2_1 = np.sum(((OC - model) / OCerr)**2)
    n = len(OC)
    k0, k1 = 1, 3  # Parameters estimated by model; offset + sin/cos terms

    bic0 = k0 * np.log(n) + chi2_0
    bic1 = k1 * np.log(n) + chi2_1

    # Preferred model (linear or sinusoidal)
    delta_BIC = bic0 - bic1
    if delta_BIC >= 10:
        best_model = 'sinusoidal'
    elif delta_BIC < 10:
        best_model = 'linear'

    # Best frequency -> period and amplitude 
    period = 1/f_best
    try:
        f_best_under, f_best_over = freq[idx_best-1], freq[idx_best+1]
        period_over, period_under = 1/f_best_under, 1/f_best_over
        period_err = max(period_over-period, period-period_under)
    except IndexError:
        period_err = np.nan
    # Sinusoidal amplitudes
    A1, A2 = ls.model_parameters(f_best)[1:]
    amp = np.sqrt(A1**2 + A2**2)
    
    # TTV strength
    scatter = MAD(OC, nan_policy='omit') / OCerr.median()
    if (delta_BIC >= 10) and (scatter >= 3):
        TTV_strength = 'strong'
    elif (delta_BIC >= 10) and (scatter >= 2):
        TTV_strength = 'weak'
    elif (delta_BIC < 10) or (scatter < 2):
        TTV_strength = 'no_TTV'

    # Plot only if delta_BIC >= 10
    if delta_BIC >= 10:
        # Phase (Plot setup)
        t_fit = np.linspace(Tmid.min(), Tmid.max(), 2000)
        phase = ((Tmid - Tmid.min()) / period) % 1 - 0.5
        phase_fit = ((t_fit - Tmid.min()) / period) % 1 - 0.5
        
        # Sort model by phase
        idx = np.argsort(phase_fit)
        phase_s = phase_fit[idx]
        model_s = ls.model(t_fit, f_best)[idx]
        
        # Break wrap-around at phase = 1 → 0
        breaks = np.where(np.diff(phase_s) < 0)[0] + 1
        phase_s = np.insert(phase_s, breaks, np.nan)
        model_s = np.insert(model_s, breaks, np.nan)
    
        # Plot
        fig, ax = plt.subplots(1, 2, figsize=(12, 3.5))
        P = 1/freq
        Pmin, Pmax = P.min(), P.max()
    
        # Plot GLS power spectrum
        ax[0].plot(1/freq, power, lw=0.5, color='black')

        # Plot FAP
        ax[0].axvline(period, lw=1, color='red', ls='-.',
                      label=f'{planet_name} peak at {1/f_best:.2f} d\nFAP={fap*100:.2g}%')
        for i, fap_ in enumerate([fap5, fap1, fap01]):
            ax[0].axhline(fap_, ls='--', lw=0.7, color=f'C{i}')

        # Plot expected Pttv
        ax[0].axvline(expected_Pttv, lw=1, color='green', ls='-.',
                      label=f'Expected superperiod at {expected_Pttv:.2f} d')
        ax[0].legend(loc='upper right')
            
        ax[0].axhline(power_best, lw=0.7, color='red', ls='-.')
        
        ax[0].set_xlabel("Period (days)"); ax[0].set_ylabel("GLS Power")
        ax[0].set_xscale("log")
        ax[0].set_xlim(Pmin / 1.05, Pmax * 1.05); ax[0].set_ylim(0, power_best * 1.5) 
    
        # Plot sinusoidal fit
        day2min = 1440
        uses_Holczer =  df.source.eq('Holczer+16').any()
        source_list = [f'Kokkori+25 ({s})' for s in ('literature', 'space', 'exoclock')] if not uses_Holczer else ['Holczer+16']
        for i, source in enumerate(source_list):
            msk = (df.source == source)
            ax[1].errorbar(phase[msk], OC[msk] * day2min, OCerr[msk] * day2min, fmt='.', color=f'C{i}', zorder=1, label=source)
        ax[1].plot(phase_s, model_s * day2min, label=f'Sinusoidal fit', color='red', lw=1, zorder=2)
        ax[1].set_xlabel("TTV Phase"); ax[1].set_ylabel("O-C (minutes)")
        ax[1].set_xlim(-0.51, 0.51)
        ax[1].legend(loc='lower right')
        
        plt.tight_layout()
        EXPORT_FILENAME = rf'fit_pics/{planet_name}_GLSfit.jpg'
        plt.savefig(EXPORT_FILENAME, dpi=300, bbox_inches="tight")
        plt.close(fig)
        del fig, ax, t_fit

    return period, period_err, amp, best_model, TTV_strength, scatter, power_relative_to_max, bic0, bic1, delta_BIC, fap, Nfreq, flag_outside_search_range, flag_insufficient_power

def expected_Pttv(planet):
    try:
        Pttv = df_final.loc[df_final['name_exoclock+holczer'] == planet, 'expected_Pttv'].iloc[0]
    except IndexError:
        Pttv = None
    return Pttv
    
# Inititate final col
df_final = df_big.copy()
df_final_newcols = ['Pttv', 'Pttv_err', 'Attv', 'best_model', 'TTV_strength', 'scatter', 'power_relative_to_max',
                    'bic0', 'bic1', 'delta_bic', 'fap', 'sampled_f',
                    'flag_outside_search_range', 'flag_insufficient_power']

df_final[df_final_newcols] = np.nan
df_final[['best_model', 'TTV_strength']] = None
df_final[['best_model', 'TTV_strength']] = df_final[['best_model', 'TTV_strength']].astype(object)

all_planets = df_clip['Planet'].dropna().astype(str).unique()
for planet in tqdm(all_planets):
    exp_Pttv = expected_Pttv(planet)
    newcol_values = lomb_scargle_analysis(df_clip, planet_name=planet,
                                          expected_Pttv=exp_Pttv)
    mask = (df_final["name_exoclock+holczer"] == planet)
    df_final.loc[mask, df_final_newcols] = newcol_values

EXPORT_FILENAME = r'data_cleaned\merged_GLSfitted.csv'
df_final.to_csv(EXPORT_FILENAME, index=False)
df_final.head(3)

100%|██████████████████████████████████████████████████████████████████████████████| 2474/2474 [02:17<00:00, 17.98it/s]


,name_exoplanet.eu,star_name,name_exoclock+holczer,T0_(BJD_TDB),T0_unc.,P_(days),P_unc.,mass,mass_error,radius,...,TTV_strength,scatter,power_relative_to_max,bic0,bic1,delta_bic,fap,sampled_f,flag_outside_search_range,flag_insufficient_power
0,HAT-P-26 b,HAT-P-26,HAT-P-26b,2.457197e+06,0.00007,4.234501,3.200000e-07,0.058500,0.007170,0.5700,...,no_TTV,0.892727,1.0,94.685567,78.505603,16.179965,0.01163,6134.0,0.0,1.0
1,HAT-P-26 d,HAT-P-26,NaN,NaN,NaN,NaN,NaN,0.020569,0.003319,0.1758,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HAT-P-27 Ac,HAT-P-27 A,NaN,NaN,NaN,NaN,NaN,0.066608,0.016082,0.3863,...,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
